# Developing function to read ref data

Currently this uses openpyxl to read off reference data from the main WSB xls.

All ref data need to be identified as named ranges in the xls (only productivity done so far).  Then pass the workbook and list of names to `get_named_ranges()`, which returns a list of pandas series.

Each range has to be scalar (i.e. a 1d array of numbers).  This means separate arrays for M F.

Currently returns a dictionary, with each range as a list.  Also have a funciton to turn this into a pd.Series, with a hierarchical index.  Good!

General idea is to use this single pd.Series as the source object for calculations.  Though actually a dictionary might not be too bad - is very intuitive.  

NB in long run has to be better to estimate models, and really just use coefficients - rather than lookup tables

These functions implemented in `read_coeffs.py`


In [1]:
from openpyxl import load_workbook
import pandas as pd

In [172]:
def get_coeffs(wb, names_list):
    '''Returns a dictionary with data from named ranges specified by names_list
    from the xls passed as wb'''

    # First load up the data - all named ranges    
    raw_dict = {}
    for name in names_list:
        raw_dict[name] = list(wb.defined_names[name].destinations)[0]

    # Now put them in a dictionary
    coeffs = {}
    for coeff in raw_dict: #must be more elegant way to iterate thru this dictionary
        sheet, rng = raw_dict[coeff]
        ws = wb[sheet]
        vals = []

        # iterate through the range if it's more than one value, otherwise just take the single value
        if type(ws[rng]) is tuple:
            for cell in ws[rng]:
                vals.append(cell[0].value)
        else:
            vals.append(cell[0].value)
        
        coeffs[coeff] = vals
    
    return coeffs


In [173]:
def make_pdseries(in_dict):
    '''Turns a dictionary into a pandas Series'''
    ser = pd.Series()
    # populate by appending to the empty series.  Note index of tuples
    for key in in_dict:
        for i,datum in enumerate(in_dict[key]):
            ser = ser.append(pd.Series({(key,i):datum}))
    # now just turn the index into a proper multi-index
    ser.index = pd.MultiIndex.from_tuples(ser.index)
    return ser

In [174]:
# USAGE
# First load in the workbook (data only, i.e. not formulas)
wb = load_workbook('data/wsi_v0.1.31.xlsx', data_only=True)

# Make the list of coefficient sets - must be the same as in the xls
names_list = ['Prod_MCS_coeffs',
              'Prod_PCS_coeffs',
              'Prod_prod_coeffs',
              'Prod_wagepcm_M_byage',
              'Prod_wagepcm_F_byage',
              'on_costs']

# Finally run it
make_pdseries(get_coeffs(wb, names_list))

Prod_MCS_coeffs       0        1.038300
                      1        5.012200
                      2       32.545900
Prod_PCS_coeffs       0       -1.044300
                      1       25.918000
                      2       31.023100
Prod_prod_coeffs      0        2.950000
                      1       -0.350000
                      2        1.190000
                      3       -0.090000
                      4        1.370000
                      5       -0.090000
                      6      -13.200000
Prod_wagepcm_M_byage  0      462.889152
                      1      462.889152
                      2      462.889152
                      3      462.889152
                      4      462.889152
                      5      462.889152
                      6      462.889152
                      7      462.889152
                      8      462.889152
                      9      462.889152
                      10     462.889152
                      11     462.889152
